In [1]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers>=4.21.0 --quiet
!pip install torch torchvision torchaudio --quiet
!pip install tqdm --quiet
!pip install datasets --quiet
!pip install datatrove --quiet

try:
    !pip install flash-attn --no-build-isolation --quiet
    print("Flash Attention がインストールされました")
except:
    print("Flash Attention のインストールに失敗 - 標準Attentionを使用します")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 79.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Flash Attention がインストールされました


In [3]:
#Google Drive内のモジュールを利用するために作業ディレクトリを変更し、pathに追加する
lightlm_path = '/content/drive/MyDrive/LightLM'

os.chdir(lightlm_path)
print(f"作業ディレクトリ: {os.getcwd()}")

if lightlm_path not in sys.path:
    sys.path.insert(0, lightlm_path)

作業ディレクトリ: /content/drive/MyDrive/LightLM


In [4]:
import torch
from transformers import AutoTokenizer
from model import Transformer, ModelConfig

# 1. 同じ設定でモデルを初期化
tokenizer_id = "HuggingFaceTB/SmolLM-360M"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token

config = ModelConfig(
    vocab_size=tokenizer.vocab_size,

    num_dims=512,
    num_heads=16,
    num_kv_heads=4,    # GQA による効率化
    num_layers=12,
    ffn_hidden_dims=512 * 4,
    # 無料版google Driveの少量すぎる保存容量と、貧弱な計算資源を考慮し、GPT-2リスペクトでさらにモデルサイズを小さく
    rmsnorm_eps=1e-6,
    rope_theta=1e5,

    context_len=512,

    use_cache=False,
    use_flash=True,    # 利用可能な場合
    use_moe=True,     # シンプル構成

    moe_num_experts=3, #一つ少なく
    moe_active_experts=1,
    moe_eps=1e-6,
    moe_aux_loss_coef=0.01,
    moe_shared_experts=1,
    use_lossfreebalance=False,
)
model = Transformer(config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [6]:
# 2. 保存済みチェックポイントをロード
checkpoint_path = lightlm_path + "/model_testing-small/model.checkpoint.epoch3_step3278_global13115.pt"
checkpoint = torch.load(checkpoint_path, map_location="cpu")

# モデル重みを読み込み
state_dict = checkpoint['model']
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("_orig_mod."):  # DDPやcompileのprefix対応
        new_state_dict[k[len("_orig_mod."):]] = v
    else:
        new_state_dict[k] = v
model.load_state_dict(new_state_dict, strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if device.type == 'cuda' else torch.float32
model.to(device, dtype=dtype)
model.eval()

print(f"Device: {device}")
print(f"Data type: {dtype}")
print(f"Model loaded on {device} with {dtype}")
print()

Device: cpu
Data type: torch.float32
Model loaded on cpu with torch.float32



In [7]:
# 3. テキスト生成の設定
text = "I am Mike. I live in"
input_ids = tokenizer(text, return_tensors="pt")["input_ids"].to(device)

# 生成パラメータ
max_tokens = 100
temperature = 0.7
top_k = 50
top_p = 0.9
repetition_penalty = 1.1
use_cache = True

print(f"Prompt: '{text}'")
print(f"Generation parameters:")
print(f"  max_tokens: {max_tokens}")
print(f"  temperature: {temperature}")
print(f"  top_k: {top_k}")
print(f"  top_p: {top_p}")
print(f"  repetition_penalty: {repetition_penalty}")
print(f"  use_cache: {use_cache}")
print()

# 4. model.generate()を使用してテキスト生成
with torch.no_grad():
    with torch.autocast(device_type=device.type, dtype=dtype):
        generated_ids = model.generate(
            x=input_ids,
            max_tokens=max_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            use_cache=use_cache
        )

# 5. 結果の表示
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

Prompt: 'I am Mike. I live in'
Generation parameters:
  max_tokens: 100
  temperature: 0.7
  top_k: 50
  top_p: 0.9
  repetition_penalty: 1.1
  use_cache: True



/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:283: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)


Generated text:
I am Mike. I live in the small town of Tewkeson, California. My father was an engineer with a passion for the earth and his passion for the natural world began when he was 21 years old. He spent his time learning about the environment, designing new products, and developing innovative technologies.
As the family moved around the neighborhood, he realized that each child was a unique individual who had different needs and interests. His parents often believed that children should learn best by understanding their own strengths and weaknesses, while


In [8]:

from google.colab import runtime
runtime.unassign()